# IESB - Graduacao - CIA035 - Malaria CNN
## Versão 2

In [1]:
# Importando as bibliotecas
import os
import time

import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, DataLoader
from torch import nn, optim

In [2]:
# Verificando o diretório com as imagens
img_dir = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images"
print(os.listdir(img_dir))

['Uninfected', 'Parasitized']


In [3]:
# Temos GPU?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


## Data Augumentation

In [4]:
# Transformações que serão aplicadas às imagens
train_transform = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(35),
                                      transforms.RandomGrayscale(p=0.02),                                      
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], 
                                                           [0.5, 0.5, 0.5])])


In [5]:
# Criando nossos conjuntos de dados com base nas imagens
train = datasets.ImageFolder(img_dir, transform=train_transform)

In [6]:
# Dividindo os dados em treino e teste
train_data, test_data = random_split(train, (20000, 7558))

In [7]:
# Definindo os loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [8]:
# Definir LeNet5

# Cálculo do tamanho da saída de cada convolução
# outputOfEachConvLayer = [(size + 2*padding - kernel_size) / stride] + 1

model = nn.Sequential(
    # Primeira convolução
    nn.Conv2d(in_channels=3, out_channels=18, kernel_size=5, padding=0, stride=1),
    # Função de ativação
    nn.Tanh(),
    # Average Pooling
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    # Segunda convolução
    nn.Conv2d(in_channels=18, out_channels=48, kernel_size=5, padding=0, stride=1),
    # Função de ativação
    nn.Tanh(),
    # Average Pooling
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    # Convertendo a imagem em estrutura plana
    nn.Flatten(),
    
    # Iniciando a rede fully connected
    # in_features = 1200
    nn.Linear(in_features=48*5*5, out_features=120),
    nn.Tanh(),
    nn.Linear(in_features=120, out_features=84),
    nn.Tanh(),
    nn.Linear(in_features=84, out_features=2)
)

model.to(device)

Sequential(
  (0): Conv2d(3, 18, kernel_size=(5, 5), stride=(1, 1))
  (1): Tanh()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(18, 48, kernel_size=(5, 5), stride=(1, 1))
  (4): Tanh()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten()
  (7): Linear(in_features=1200, out_features=120, bias=True)
  (8): Tanh()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Tanh()
  (11): Linear(in_features=84, out_features=2, bias=True)
)

In [9]:
# Temos que definir a função de erro e o otimizador (que vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

error_function.to(device)

CrossEntropyLoss()

In [10]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 10

# Colocando o modelo em modo de treinamento
model.train()


# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Monitorando o training loss
    train_loss = 0.0
    
    # Obtendo dados e respostas
    for data, target in train_loader:
        
        # Enviando os dados para GPU, se existir
        data, target = data.to(device), target.to(device)
    
        # Foward Propagation (passando os dados de treino pela rede)
        outputs = model(data)
        # Calculando o erro
        loss = error_function(outputs, target)
       
        # Back Propagation
        # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
        optimizer.zero_grad()
        # Calcular os novos pesos
        loss.backward()
        # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
        optimizer.step()
        
        # Atualizando o training loss
        train_loss += loss.item() * data.size(0)
        
    # Calculando a média de erro por epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(i+1, train_loss))

Epoch: 1 	Training Loss: 0.610153
Epoch: 2 	Training Loss: 0.569974
Epoch: 3 	Training Loss: 0.508284
Epoch: 4 	Training Loss: 0.402826
Epoch: 5 	Training Loss: 0.339283
Epoch: 6 	Training Loss: 0.309892
Epoch: 7 	Training Loss: 0.286090
Epoch: 8 	Training Loss: 0.274958
Epoch: 9 	Training Loss: 0.267036
Epoch: 10 	Training Loss: 0.258260


In [11]:
# Variaveis para controlar os acertos das previsões da rede
# e  calcular a acurácia
correct = 0
total = 0

# Vamos colocar o modelo em modo de avaliação/teste
model.eval()

# Obtendo dados e respostas
for data, target in test_loader:
    
    # Enviando os dados para GPU, se existir
    data, target = data.to(device), target.to(device)    
    
    output = model(data)
    
    for index, i in enumerate(output):
        if torch.argmax(i) == target[index]:
            correct += 1
        total += 1

In [12]:
print('Accuracy: ', round(correct/total, 3))

Accuracy:  0.895


## Tentando com uma nova rede CNN

In [13]:
# Transformações que serão aplicadas às imagens
new_transform = transforms.Compose([transforms.Resize((120, 120)),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomVerticalFlip(),
                                    transforms.RandomRotation(35),
                                    transforms.RandomGrayscale(p=0.02),                                      
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5, 0.5, 0.5], 
                                                         [0.5, 0.5, 0.5])])

# Criando nossos conjuntos de dados com base nas imagens
train = datasets.ImageFolder(img_dir, transform=new_transform)

# Dividindo os dados em treino e teste
train_data, test_data = random_split(train, (20000, 7558))

# Definindo os loaders
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=True)

In [14]:
# Definir nova rede CNN

# Cálculo do tamanho da saída de cada convolução
# outputOfEachConvLayer = [(size + 2*padding - kernel_size) / stride] + 1

model = nn.Sequential(
    # 120 x 120, 3 canais
    
    # Primeira convolução
    nn.Conv2d(in_channels=3, out_channels=18, kernel_size=5, padding=2, stride=1),
    # Função de ativação
    nn.ReLU(),
    # Max Pooling
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    # 60 x 60, 18 canais
    
    # Segunda convolução
    nn.Conv2d(in_channels=18, out_channels=48, kernel_size=5, padding=2, stride=1),
    # Função de ativação
    nn.ReLU(),
    # Max Pooling
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    # 30 x 30, 48 canais
    
    # Terceira convolução
    nn.Conv2d(in_channels=48, out_channels=144, kernel_size=5, padding=2, stride=1),
    # Função de ativação
    nn.ReLU(),
    # Max Pooling
    nn.MaxPool2d(kernel_size=2, stride=2),  
    
    # 15 x 15, 144 canais
    
    # Convertendo a imagem em estrutura plana
    nn.Flatten(),
    
    # Iniciando a rede fully connected
    # in_features = 32400
    nn.Linear(in_features=144*15*15, out_features=5200),
    nn.ReLU(),
    nn.Linear(in_features=5200, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=2)
)

model.to(device)

Sequential(
  (0): Conv2d(3, 18, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(18, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(48, 144, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=32400, out_features=5200, bias=True)
  (11): ReLU()
  (12): Linear(in_features=5200, out_features=256, bias=True)
  (13): ReLU()
  (14): Linear(in_features=256, out_features=64, bias=True)
  (15): ReLU()
  (16): Linear(in_features=64, out_features=2, bias=True)
)

In [15]:
# Temos que definir a função de erro e o otimizador (que vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

error_function.to(device)

CrossEntropyLoss()

In [16]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 10

# Colocando o modelo em modo de treinamento
model.train()


# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Monitorando o training loss
    train_loss = 0.0
    
    # Obtendo dados e respostas
    for data, target in train_loader:
        
        # Enviando os dados para GPU, se existir
        data, target = data.to(device), target.to(device)
    
        # Foward Propagation (passando os dados de treino pela rede)
        outputs = model(data)
        # Calculando o erro
        loss = error_function(outputs, target)
       
        # Back Propagation
        # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
        optimizer.zero_grad()
        # Calcular os novos pesos
        loss.backward()
        # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
        optimizer.step()
        
        # Atualizando o training loss
        train_loss += loss.item() * data.size(0)
        
    # Calculando a média de erro por epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(i+1, train_loss))

Epoch: 1 	Training Loss: 0.515424
Epoch: 2 	Training Loss: 0.213677
Epoch: 3 	Training Loss: 0.157310
Epoch: 4 	Training Loss: 0.145359
Epoch: 5 	Training Loss: 0.141279
Epoch: 6 	Training Loss: 0.133186
Epoch: 7 	Training Loss: 0.128593
Epoch: 8 	Training Loss: 0.124116
Epoch: 9 	Training Loss: 0.123095
Epoch: 10 	Training Loss: 0.123640


In [17]:
# Variaveis para controlar os acertos das previsões da rede
# e  calcular a acurácia
correct = 0
total = 0

# Vamos colocar o modelo em modo de avaliação/teste
model.eval()

# Obtendo dados e respostas
for data, target in test_loader:
    
    # Enviando os dados para GPU, se existir
    data, target = data.to(device), target.to(device)    
    
    output = model(data)
    
    for index, i in enumerate(output):
        if torch.argmax(i) == target[index]:
            correct += 1
        total += 1
        
print('Accuracy: ', round(correct/total, 3))

Accuracy:  0.954
